## 금융 문서 검색 시스템: 사용자 의도 명확화 및 멀티쿼리 RAG 에이전트 구축 연습

목표
- 사용자의 질문 의도를 분석하고, 불명확한 경우 구체적인 질문으로 되물어보는 시스템을 구축합니다.
- 의도가 명확해지면 보다 풍부한 RAG 검색을 위한 멀티쿼리를 생성하는 에이전트를 만듭니다.
- 금융 도메인 문서 검색에 최적화된 쿼리 변환 및 처리 과정을 학습합니다.

요구사항
- `State` TypedDict에 `messages`, `user_intent`, `search_queries`, `intent_clarity`를 정의하세요.
- `ChatOpenAI` 모델(gpt-4.1, temperature=0)을 사용하여 의도 파악 및 쿼리 생성을 수행하세요.
- 의도가 불명확한 경우 사용자에게 구체적인 질문을 하고, 명확한 경우 3-5개의 검색 쿼리를 생성하세요.
- START → intent_analysis → (clarify_intent OR generate_queries) → END 경로를 구성하세요.

도메인 정보
- 금융 문서 검색 시스템 (주식, 채권, 펀드, 보험, 대출, 투자 상품 등)
- 3가지 프롬프트 지침:
  1. **정확성 우선**: 금융 정보의 정확성과 신뢰성을 최우선으로 합니다.
  2. **규제 준수**: 금융 관련 법규와 규정을 반드시 고려합니다.
  3. **리스크 명시**: 투자나 금융 상품의 위험 요소를 명확히 전달합니다.

In [ ]:
from dotenv import load_dotenv
from langchain_teddynote import logging

load_dotenv(override=True)

# 프로젝트 이름
logging.langsmith("LangGraph-Exercises")

In [ ]:
# Part 1 준비 코드
from typing import Annotated, List, Optional
from typing_extensions import TypedDict
from dotenv import load_dotenv

from langchain_teddynote import logging
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from pydantic import BaseModel, Field

### 1. 모델을 생성해 주세요.

- 사용 모델: `ChatOpenAI`, 모델명: `gpt-4.1`, 온도: `0`

In [ ]:
# 실습 코드
# TODO: LLM 모델을 정의해 주세요.
llm = # 코드 입력

### 2. State 및 데이터 모델 정의

- 그래프에서 사용할 State를 정의해 주세요.
- 사용자 의도와 검색 쿼리를 관리하기 위한 Pydantic 모델도 정의하세요.

In [ ]:
# 실습 코드(제공되는 코드)
# TODO: 의도 분석 결과를 담을 Pydantic 모델을 정의해 주세요.
class IntentAnalysis(BaseModel):
    is_clear: bool = Field(description="의도가 명확한지 여부")
    intent_category: str = Field(description="의도 카테고리 (투자상품, 대출, 보험 등)")
    clarification_questions: List[str] = Field(description="되물어볼 질문들")
    confidence_score: float = Field(description="의도 파악 신뢰도 (0-1)")


# TODO: 멀티쿼리 생성 결과를 담을 Pydantic 모델을 정의해 주세요.
class MultiQueryGeneration(BaseModel):
    original_query: str = Field(description="원본 쿼리")
    search_queries: List[str] = Field(description="생성된 검색 쿼리 리스트")
    query_rationale: List[str] = Field(description="각 쿼리 생성 근거")


# TODO: 그래프에서 사용할 State를 정의해 주세요.
class State(TypedDict):
    messages: Annotated[list, add_messages]
    user_intent: Optional[dict]  # 사용자 의도 분석 결과
    search_queries: Optional[List[str]]  # 생성된 검색 쿼리들
    intent_clarity: bool  # 의도 명확성 여부

### 3. 의도 분석 노드 작성

- 사용자의 질문에서 의도를 분석하는 노드를 작성해 주세요.
- 금융 도메인에 특화된 의도 분석을 수행하고, 불명확한 경우 구체적인 질문을 준비하세요.
- IntentAnalysis 모델을 도구로 바인딩하여 구조화된 결과를 반환하세요.

In [ ]:
# 실습 코드
# TODO: 의도 분석을 위한 시스템 프롬프트를 작성해 주세요.
INTENT_ANALYSIS_PROMPT = """당신은 금융 문서 검색 시스템의 의도 분석 전문가입니다.
사용자의 질문을 분석하여 다음을 판단해주세요:

1. 의도가 명확한지 여부 (is_clear: True/False)
2. 금융 카테고리 분류 (투자상품, 대출, 보험, 예적금, 카드, 외환, 세금, 부동산금융 등)
3. 불명확한 경우 구체적인 되물어볼 질문들 (최대 3개)
4. 의도 파악 신뢰도 (0.0-1.0)

금융 정보의 정확성과 규제 준수를 고려하여 신중하게 분석하세요.
애매한 질문의 경우 반드시 구체적인 정보를 요청하세요.

예시 명확한 질문: "삼성전자 주식의 최근 3개월 주가 동향을 알려주세요"
예시 불명확한 질문: "좋은 투자 방법 알려주세요"
"""

# TODO: 의도 분석 노드를 구현해 주세요.
def analyze_intent(state: State):
    # 코드 입력: IntentAnalysis를 도구로 바인딩
    llm_with_intent_tool = # 코드 입력
    
    # 코드 입력: 시스템 메시지와 사용자 메시지 구성
    messages = [
        # 코드 입력
    ]
    
    # 코드 입력: LLM 호출 및 결과 처리
    response = # 코드 입력
    
    # 코드 입력: 도구 호출 결과 추출
    if response.tool_calls:
        intent_result = # 코드 입력
        return {
            "messages": [response],
            "user_intent": intent_result,
            "intent_clarity": intent_result["is_clear"]
        }
    
    return {"messages": [response], "intent_clarity": False}

### 4. 의도 명확화 노드 작성

- 사용자의 의도가 불명확한 경우 구체적인 질문을 하는 노드를 작성해 주세요.
- 금융 도메인의 특성을 고려하여 사용자가 더 구체적인 정보를 제공하도록 유도하세요.

In [ ]:
# 실습 코드
# TODO: 의도 명확화 노드를 구현해 주세요.
def clarify_intent(state: State):
    user_intent = state["user_intent"]
    
    if user_intent and user_intent.get("clarification_questions"):
        # 코드 입력: 되물어볼 질문들을 포맷팅
        questions = # 코드 입력
        
        clarification_message = f"""질문을 보다 정확히 이해하기 위해 몇 가지 추가 정보가 필요합니다:

{questions}

위 정보를 제공해 주시면 더 정확하고 유용한 답변을 드릴 수 있습니다."""
    else:
        clarification_message = "질문의 의도를 파악하기 어렵습니다. 더 구체적인 질문을 해주시겠어요?"
    
    # 코드 입력: AIMessage로 응답 반환
    return {"messages": [# 코드 입력]}

### 5. 멀티쿼리 생성 노드 작성

- 사용자의 의도가 명확한 경우 RAG 검색을 위한 여러 개의 검색 쿼리를 생성하는 노드를 작성해 주세요.
- 금융 도메인에 특화된 다양한 관점의 검색 쿼리를 생성하세요.

In [ ]:
# 실습 코드
# TODO: 멀티쿼리 생성을 위한 시스템 프롬프트를 작성해 주세요.
MULTI_QUERY_PROMPT = """당신은 금융 문서 검색을 위한 쿼리 생성 전문가입니다.
사용자의 질문을 바탕으로 보다 풍부하고 정확한 검색을 위해 3-5개의 다양한 검색 쿼리를 생성해주세요.

다음 관점을 고려하여 쿼리를 생성하세요:
1. 핵심 키워드 중심의 직접적 검색
2. 관련 금융 상품이나 서비스 검색
3. 규제나 법률적 관점의 검색
4. 리스크나 주의사항 관점의 검색
5. 대안이나 비교 상품 검색

각 쿼리는 구체적이고 검색 가능해야 하며, 중복을 피해야 합니다.
생성 근거도 함께 제공해주세요.
"""

# TODO: 멀티쿼리 생성 노드를 구현해 주세요.
def generate_multi_queries(state: State):
    # 코드 입력: MultiQueryGeneration을 도구로 바인딩
    llm_with_query_tool = # 코드 입력
    
    # 코드 입력: 원본 사용자 메시지 추출
    original_query = None
    for msg in state["messages"]:
        if isinstance(msg, HumanMessage):
            original_query = msg.content
            break
    
    # 코드 입력: 시스템 메시지와 쿼리 구성
    messages = [
        SystemMessage(content=MULTI_QUERY_PROMPT),
        HumanMessage(content=f"사용자 질문: {original_query}")
    ]
    
    # 코드 입력: LLM 호출 및 결과 처리
    response = # 코드 입력
    
    # 코드 입력: 도구 호출 결과 추출
    if response.tool_calls:
        query_result = # 코드 입력
        
        # 검색 쿼리 결과를 포맷팅하여 메시지 생성
        queries_text = "\n".join([f"{i+1}. {q}" for i, q in enumerate(query_result["search_queries"])])
        result_message = f"""✅ 다음과 같은 검색 쿼리들을 생성했습니다:

{queries_text}

이 쿼리들을 사용하여 포괄적인 문서 검색을 수행할 수 있습니다."""
        
        return {
            "messages": [AIMessage(content=result_message)],
            "search_queries": query_result["search_queries"]
        }
    
    return {"messages": [AIMessage(content="쿼리 생성에 실패했습니다.")]}

### 6. 그래프 생성 및 조건부 엣지 설정

- 그래프를 생성하고 조건부 엣지를 설정해 주세요.
- 의도가 명확하지 않으면 명확화 노드로, 명확하면 멀티쿼리 생성 노드로 이동하도록 구성하세요.

In [ ]:
# 실습 코드
# TODO: 상태 결정 함수를 구현해 주세요.
def determine_next_step(state: State):
    # 코드 입력: intent_clarity 값에 따라 다음 단계 결정
    if state.get("intent_clarity", False):
        return # 코드 입력
    else:
        return # 코드 입력

# TODO: 그래프를 생성하고 노드와 엣지를 구성해 주세요.
builder = # 코드 입력

# 노드 추가
builder.# 코드 입력
builder.# 코드 입력
builder.# 코드 입력

# 엣지 구성
builder.# 코드 입력  # START에서 의도 분석으로

# 조건부 엣지 설정
builder.# 코드 입력

# 종료 엣지
builder.# 코드 입력
builder.# 코드 입력

# 그래프 컴파일
graph = builder.# 코드 입력

### 7. 그래프 시각화

- 컴파일한 그래프를 시각화 하세요.

In [ ]:
# 실습 코드
# TODO: 그래프를 시각화 하세요.

from langchain_teddynote.graphs import visualize_graph

# 코드 입력

### 8. 그래프 실행 및 테스트

- 금융 도메인의 다양한 질문으로 그래프를 테스트해보세요.
- 명확한 질문과 불명확한 질문을 모두 시도해보세요.

아래의 코드를 실행하여 결과를 확인하세요

In [ ]:
# TODO: 불명확한 질문으로 테스트해보세요.
from langchain_teddynote.messages import stream_graph, random_uuid
from langchain_core.runnables import RunnableConfig

config = RunnableConfig(configurable={"thread_id": random_uuid()})

# 불명확한 질문 예시
unclear_inputs = {
    "messages": [HumanMessage(content="좋은 투자 방법 알려주세요")],
    "intent_clarity": False,
}

print("=== 불명확한 질문 테스트 ===")
stream_graph(graph, inputs=unclear_inputs, config=config)

In [ ]:
# 불명확한 질문 예시
unclear_inputs = {
    "messages": [
        HumanMessage(
            content="주식에 대한 투자 방법을 찾고 있어요. 투자 기간은 중기, 수익률은 10% 이상, 위험은 5% 이하로 해주세요."
        )
    ],
    "intent_clarity": False,
}

print("=== 불명확한 질문 테스트 ===")
stream_graph(graph, inputs=unclear_inputs, config=config)

In [ ]:
# TODO: 명확한 질문으로 테스트해보세요.

# 명확한 질문 예시
clear_inputs = {
    "messages": [
        HumanMessage(
            content="삼성전자 주식의 최근 6개월 주가 변동과 배당 정보를 알려주세요"
        )
    ],
    "intent_clarity": False,
}

print("\n=== 명확한 질문 테스트 ===")
stream_graph(graph, inputs=clear_inputs, config=config)

In [ ]:
# 추가 테스트 - 다양한 금융 도메인 질문들
test_questions = [
    "주택담보대출 금리 비교해주세요",
    "연금보험 상품 추천해주세요",
    "비트코인 ETF 투자 위험성에 대해 알려주세요",
    "ISA 계좌 개설 조건과 세제 혜택을 설명해주세요",
]

for i, question in enumerate(test_questions, 1):
    print(f"\n=== 테스트 {i}: {question} ===")
    test_inputs = {
        "messages": [HumanMessage(content=question)],
        "intent_clarity": False,
    }
    stream_graph(graph, inputs=test_inputs, config=config)